In [1]:
import os

import jax
import jax.numpy as jnp
import numpy as np
from scipy import optimize

from vibrojet.basis_utils import ContrBasis, HermiteBasis, generate_prod_ind
from vibrojet.keo import Gmat, com
from vibrojet.potentials import h2co_AYTY
from vibrojet.taylor import deriv_list

jax.config.update("jax_enable_x64", True)

Equilibrium geometry

In [2]:
vmin = optimize.minimize(h2co_AYTY.poten, [1, 1, 1, 2, 2, np.pi])
r0 = vmin.x
v0 = vmin.fun
print("equilibrium coordinates:", r0)
print("min of the potential:", v0)

equilibrium coordinates: [1.19999999 1.1        1.1        2.13       2.13       3.14159264]
min of the potential: 1.735341153700765e-11


Coordinate mapping

In [3]:
masses = [12.0, 15.990526, 1.00782505, 1.00782505]  # masses of C, O, H, H

ncoo = len(r0)


# @eckart(r0, masses)
@com(masses)
def internal_to_cartesian(internal_coords):
    rCO, rCH1, rCH2, aOCH1, aOCH2, tau = internal_coords
    xyz = [
        [0.0, 0.0, 0.0],
        [0.0, 0.0, rCO],
        [
            rCH1 * jnp.sin(aOCH1) * jnp.cos(tau * 0.5),
            -rCH1 * jnp.sin(aOCH1) * jnp.sin(tau * 0.5),
            rCH1 * jnp.cos(aOCH1),
        ],
        [
            rCH2 * jnp.sin(aOCH2) * jnp.cos(tau * 0.5),
            rCH2 * jnp.sin(aOCH2) * jnp.sin(tau * 0.5),
            rCH2 * jnp.cos(aOCH2),
        ],
    ]
    return jnp.array(xyz, dtype=jnp.float64)

Generate expansions for the KEO and PES

In [4]:
max_pow = 4
powers = [np.arange(max_pow + 1)] * ncoo
deriv_ind, deriv_mind = next(
    generate_prod_ind(powers, select=lambda ind: np.sum(ind) <= max_pow)
)

print("max expansion power:", max_pow)
print("number of expansion terms:", len(deriv_ind))

max expansion power: 4
number of expansion terms: 210


In [5]:
poten_file = "_h2co_poten_coefs.npy"
if os.path.exists(poten_file):
    print(f"load potential expansion coefs from file {poten_file}")
    poten_coefs = np.load(poten_file)
else:
    poten_coefs = deriv_list(h2co_AYTY.poten, deriv_ind, r0, if_taylor=True)
    np.save(poten_file, poten_coefs)

load potential expansion coefs from file _h2co_poten_coefs.npy


In [6]:
Gmat_file = "_h2co_Gmat_coefs.npy"
if os.path.exists(Gmat_file):
    print(f"load G-matrix expansion coefs from file {Gmat_file}")
    Gmat_coefs = np.load(Gmat_file)
else:
    Gmat_coefs = deriv_list(
        lambda x: Gmat(x, masses, internal_to_cartesian), deriv_ind, r0, if_taylor=True
    )
    np.save(Gmat_file, Gmat_coefs)

load G-matrix expansion coefs from file _h2co_Gmat_coefs.npy


Define linear mapping between coordinates of Hermite functions and internal valence coordinates

In [7]:
mask = deriv_ind != 0
ind0 = np.where(mask.sum(axis=1) == 0)[0][0]
mu = np.diag(Gmat_coefs[ind0])[:ncoo]

ind2 = np.array(
    [
        np.where((mask.sum(axis=1) == 1) & (deriv_ind[:, icoo] == 2))[0][0]
        for icoo in range(ncoo)
    ]
)
freq = poten_coefs[ind2] * 2

lin_a = jnp.sqrt(jnp.sqrt(mu / freq))
lin_b = r0

print("x->r linear mapping parameters 'a':", lin_a)
print("x->r linear mapping parameters 'b':", lin_b)

# x->r linear mapping function
x_to_r_map = lambda x, icoo: lin_a[icoo] * x + lin_b[icoo]
jac_x_to_r_map = lambda x, icoo: np.ones_like(x) * lin_a[icoo]

x->r linear mapping parameters 'a': [0.05235088 0.11034365 0.11034365 0.15434588 0.15434588 0.3003904 ]
x->r linear mapping parameters 'b': [1.19999999 1.1        1.1        2.13       2.13       3.14159264]


Define primitive basis

In [8]:
nbas = [10] * ncoo
npoints = [80] * ncoo

p0, p1, p2, p3, p4, p5 = [
    HermiteBasis(
        icoo,
        nbas[icoo],
        npoints[icoo],
        lambda x: x_to_r_map(x, icoo),
        r0[icoo],
        deriv_ind[:, icoo],
    )
    for icoo in range(ncoo)
]

Build contracted basis

In [9]:
# test 1D shit
c0, c1, c2, c3, c4, c5 = [
    ContrBasis(
        (icoo,), (p0, p1, p2, p3, p4, p5), lambda _: True, Gmat_coefs, poten_coefs
    )
    for icoo in range(ncoo)
]

# test coupling shit
c12 = ContrBasis(
    (1, 2), (p0, c1, c2, p3, p4, p5), lambda _: True, Gmat_coefs, poten_coefs
)

3498.4005430664756 [    0.          1767.49826544  3516.20988539  5249.45524218
  6972.82534736  8710.87648757 10566.40021498 12531.54185015
 14998.00989534 20431.50042489]
3754.100055613889 [    0.          2877.97162589  5734.12572548  8622.19292086
 11576.24234791 14682.33717759 18077.95305985 21796.68277158
 28565.75690689 45995.46037177]
3754.100056503732 [    0.          2877.97162626  5734.12572571  8622.19292046
 11576.24234645 14682.33717391 18077.95305288 21796.68275781
 28565.75687903 45995.46032979]
3439.8386374881943 [    0.          1504.00373554  3008.09100223  4512.26469289
  6016.52858298  7520.88536795  9025.41834462 10535.49831869
 12042.95127454 13552.30491157]
3439.8386363352856 [    0.          1504.00373377  3008.09099864  4512.26468744
  6016.52857563  7520.88535865  9025.41833339 10535.49830526
 12042.9512593  13552.30489777]
3370.6659956656254 [    0.          1229.29736327  2475.70531137  3738.29867426
  5016.30579807  6308.99769789  7619.60750169  8940.69553